## 本段程序简介（实际操作起来感觉非常耗时，分析原因在于调用的内置函数，待优化ing）
> 功能：使用自动化、多线程，同时处理24个集合。<br>
> 目标：将24个集合（包含了24个小时的发文内容）中的每条记录进行清洗 & 信息提取

## 数据清洗（总结下来需要清洗的对象）
> 停用词过滤
> * 使用停用词表
标点符号过滤（包括`标点符号、标签、空字符串`）
> * 使用标点符号表
表情过滤（表情的一般表示：[表情]）
> * 使用正则表达式即可简单过滤
广告（http://xxxxx)：视频、音频、其他链接
> * 使用正则表达式删除


> 无效词过滤
> * 阈值为3（出现次数小于3的）
> * 思路：是否可以把词频图做出来，取中间段？

> 其他
> * 缺失值、补全缺失值
> * 乱码数据、修正乱码
> * 异常数据、格式规范化

**后续步骤**（`提取`的操作，都会将对应的value放到新字段中）
* Step1. 提取人名、地名
* Step2. 去除网址
* Step3. 提取话题，删除话题
* Step4. 提取用户，删除用户
* Step5. 去除表情
* Step6. 提取关键词（不用停用词表进行分词了）

In [1]:
import pymongo
con_str = "localhost:27017"
db = pymongo.MongoClient(con_str, serverSelectionTimeoutMS = 10000)
try:
    # print(db.server_info())
    
    # 查询数据库中所有数据库的名称
    db_names = db.list_database_names()
    print(db_names)
except Exception:
    print("connect failed!")
    
targetDB = 'db_Weibo_SH0331'

if targetDB in db_names:
    db_Weibo_SH0331 = db['db_Weibo_SH0331']
    print(targetDB + ": 已连接！")

['admin', 'config', 'db_Weibo_SH0331', 'db_Weibo_SH_0331_All', 'db_tvcgR3', 'db_weiboDB', 'local', 'test', 'weiboInfoAll']
db_Weibo_SH0331: 已连接！


## 两种方式实现（均已实现）：
1. 使用自己定义停用词表；
> * 已实现，下面是功能函数，方便以后使用
2. 使用jieba的analyse模块；(本段程序采用此方法）
> * 默认提取20个关键词
> * 默认提取权重靠前的人名
> * 默认提取权重靠前的地名

### 【opt.1】 使用jieba进行分词（1/2 读取停用词表）

In [2]:
import codecs  # 编解码器模块，这里用于utf-8文件的读取

# 停用词词表文件读取，函数传入文件路径，返回停用词集合
def read_stopwords(filename):
    # 使用codecs库来打开文件，并以utf-8的文件格式读取文件中的每一行
    lines = codecs.open(filename, 'r', encoding='utf-8').readlines()
    stopwords = set()  # 保存停用词的集合
    for line in lines:  # 遍历读取词表中的每一行
        line = line.strip()  # 将该行的首尾空字符去掉后
        stopwords.add(line)  # 添加到集合中

    return stopwords  # 返回停用词集合

### 【opt.1】使用jieba进行分词（1/2 分词处理函数）

In [3]:
import jieba  # 导入jieba模块

def sentencesCutting(sent, stopwords):
    # cutInit = nltk.word_tokenize(sent)  # 初始分词结果
    seg_result = jieba.cut(sent)  # 调用默认的精准模式进行分词

    # 去除所有停用词
    cutPun8Sw = [word for word in seg_result if word not in stopwords]   # 去除中文停用词

    return cutPun8Sw

### 【opt.2】分词 & 提取（`人名`，`地名`，`话题`，`用户`、除去话题和用户之外的`分词结果`）

In [2]:
import jieba
import jieba.analyse  # 导入jieba的analyse模块

import re
import sys

def filterSentences(targetCol, fields):    
    
    # 提取/过滤（网址、话题、用户名）
    patternLink = re.compile(r'((http|https|ftp|ftps):\/\/)?([a-zA-Z0-9-]+\.){1,5}(com|cn|net|org|hk|tw)((\/(\w|-)+(\.([a-zA-Z]+))?)+)?(\/)?(\??([\.%:a-zA-Z0-9_-]+=[#\.%:a-zA-Z0-9_-]+(&amp;)?)+)?')
    patternTopic = re.compile(r"\#(.+?)\#")
    patternUsers = re.compile(r'@([\u4e00-\u9fa5\w\-]+)')
    
    exec(targetCol + " = db_Weibo_SH0331['" + targetCol+ "']")
    exec("recs = " + targetCol + ".find()")    
    
    loc = locals()
    for c in loc['recs']: 
        id = c['_id']
        
        # 操作博文主题/内容 ------------------------------
        title = c['title8content'] 
        # 操作博文原始内容 ------------------------------
        orgCon = c['orgContent']
        
        plusTxt = title + orgCon
        
        plusTxt = re.sub(patternLink, "", plusTxt) # 去除网址
        plusTxt = re.sub(r"\[(.+?)\]", "", plusTxt)  # 去除表情
        personsName = jieba.analyse.extract_tags(plusTxt,allowPOS=('nr',))  # ●
        placesName = jieba.analyse.extract_tags(plusTxt,allowPOS=('ns',))  # ●
        resTopic = patternTopic.findall(plusTxt)  # 提取plusTxt中的话题 # ●
        # plusTxt = re.sub(patternTopic, "", plusTxt)  # 删除plusTxt中的话题
        resUsers = patternUsers.findall(plusTxt)  # 提取plusTxt中的用户 # ●
        # plusTxt = re.sub(patternUsers, "", plusTxt)  # 删除plusTxt中的用户
        kWords = jieba.analyse.extract_tags(plusTxt) # ●
        
        f1 = fields[0]  # personsName
        f2 = fields[1]  # placesName
        f3 = fields[2]  # resTopic
        f4 = fields[3]  # resUsers
        f5 = fields[4]  # kWords
        
        '''
        if not c.get(f1):  # 判断当前字段的值是否为空，等同于( if c['titleCutting'] == [] )
            exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f1 + "': personsName}})")
        
        if not c.get(f2):
            exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f2 + "': placesName}})")

        if not c.get(f3):
            exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f3 + "': resTopic}})")
            
        if not c.get(f4):
            exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f4 + "': resUsers}})")
            
        if not c.get(f5):
            exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f5 + "': kWords}})")
        '''
        
        exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f1 + "': personsName}})")
        exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f2 + "': placesName}})")
        exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f3 + "': resTopic}})")
        exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f4 + "': resUsers}})")
        exec(targetCol + ".update_one({'_id': id}, {'$set': {'" + f5 + "': kWords}})")

In [3]:
import threading
import time

class myThread(threading.Thread):
    def __init__(self, threadName, targetCol, fields):
        threading.Thread.__init__(self)
        self.threadName = threadName
        self.targetCol = targetCol
        self.fields = fields
        
    def run(self):
        print ("开始线程：" + str(self.threadName))
        filterSentences(self.targetCol, self.fields)
        print ("退出线程：" + str(self.threadName))

In [ ]:
# 用继承实现的线程（对比实验：用多线程操作）
import time

if __name__ == '__main__':
    
    startTime = time.time()
    
    # 循环处理24个集合
    subCol24 = list()
    subCol24.append(["col_weibo_timePeriod_" + str(period) for period in range(0, 24)])
    colPackages = subCol24[0]
    
    fields = ['personsName', 'placesName', 'resTopic', 'resUsers', 'kWords']
    
    threads = []
    for i in range(0, len(colPackages)):
        t = myThread("col" + str(i), colPackages[i], fields)
        t.start()
        threads.append(t)
        
    [thread.join() for thread in threads]
    print ("退出主线程")
    
    endTime = time.time()    
    print("Time consuming(s): %.2f" % (endTime - startTime))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\t\AppData\Local\Temp\jieba.cache


开始线程：col0
开始线程：col1
开始线程：col2开始线程：col3

开始线程：col4
开始线程：col5
开始线程：col6
开始线程：col7
开始线程：col8
开始线程：col9
开始线程：col10
开始线程：col11
开始线程：col12
开始线程：col13
开始线程：col14开始线程：col15

开始线程：col16
开始线程：col17
开始线程：col18
开始线程：col19
开始线程：col20
开始线程：col21
开始线程：col22
开始线程：col23


Loading model cost 2.361 seconds.
Prefix dict has been built successfully.
